In [ ]:
import numpy as np
import pandas as pd
import os

## form dataframe with tmdb desc, poster, and script text

In [ ]:
df = pd.read_csv(
    "..\\database\\dataset_film_scripts\\springfield_movie_scripts_2023_01_13_clean.csv",
    index_col=[0],
)
df = df[df["tmdb_id"] > 0]

dff = pd.read_csv("..\\database\\dataset_tmdb\\df_tmdb_description.csv", index_col=[0])

poster_list = os.listdir("..\\database\\dataset_tmdb\\posters")

df_spaces = pd.read_csv("..\\database\\dataset_spaces_upload.csv", index_col=[0])
df_spaces = df_spaces.drop_duplicates(subset="imdb_id", keep=False)
df_spaces = df_spaces[["imdb_id", "genre", "average_rating", "num_votes"]]

In [ ]:
# drop all duplicates from df and dff
df = df.drop_duplicates(subset=["tmdb_id"], keep=False)
dff = dff.drop_duplicates(subset=["tmdb_id"], keep=False)

dff = dff[~dff["tmdb_description"].isna()]

In [ ]:
# find subset of tmdb_ids that are in posters, desc, and script
tmdb_id_posters = [int(j[5:-4]) for j in poster_list]
tmdb_id_desc = list(dff["tmdb_id"])
tmdb_id_script = list(df["tmdb_id"])

# find tmdb ids that have poster, tmdb description, and film script
tmdb_id_combined = [j for j in tmdb_id_desc if j in tmdb_id_posters]
tmdb_id_combined = [j for j in tmdb_id_combined if j in tmdb_id_script]
len(tmdb_id_combined)

32173

In [ ]:
# drop rows that are not in tmdb_id_combined
dff = dff[dff["tmdb_id"].isin(tmdb_id_combined)]
df = df[df["tmdb_id"].isin(tmdb_id_combined)]

In [ ]:
poster_list = [
    poster_list[j]
    for j in range(len(tmdb_id_posters))
    if tmdb_id_posters[j] in tmdb_id_combined
]
tmdb_id_posters = [int(j[5:-4]) for j in poster_list]
df_posters = pd.DataFrame(
    list(zip(tmdb_id_posters, poster_list)), columns=["tmdb_id", "poster_img"]
)

In [ ]:
df_embedding = df[
    ["imdb_id", "tmdb_id", "movie_title", "movie_year", "script_text"]
].copy()
df_embedding = df_embedding.join(dff.set_index("tmdb_id"), on="tmdb_id", how="left")
df_embedding = df_embedding.join(
    df_posters.set_index("tmdb_id"), on="tmdb_id", how="left"
)
df_embedding = df_embedding.join(
    df_spaces.set_index("imdb_id"), on="imdb_id", how="left"
)

df_embedding["average_rating"] = df_embedding["average_rating"].fillna(-1)
df_embedding["num_votes"] = df_embedding["num_votes"].fillna(-1)
df_embedding["genre"] = df_embedding["genre"].fillna("None Listed")
df_embedding.loc[
    df_embedding[df_embedding["genre"] == "\\N"].index, "genre"
] = "None Listed"
df_embedding["script_id"] = df_embedding.index
df_embedding = df_embedding.reset_index(drop=True)

df_embedding = df_embedding[
    [
        "movie_title",
        "movie_year",
        "genre",
        "average_rating",
        "num_votes",
        "script_text",
        "tmdb_description",
        "poster_img",
        "tmdb_id",
        "imdb_id",
        "script_id",
    ]
]

df_embedding.to_csv("df_embeddings.csv")